In [2]:
## PART1 : SELF MADE NEURAL NETWORK

In [3]:
import os
import numpy as np
import tarfile
from tqdm import tqdm
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
with tarfile.open('cifar-10-python.tar.gz', 'r:gz') as tar_ref:
    tar_ref.extractall('cifar-data')


--2023-04-29 18:06:14--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  51.0MB/s    in 3.5s    

2023-04-29 18:06:18 (46.5 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]



In [4]:
import numpy as np 


def correlate2DValid(A,B):
  b1,b2=B.shape
  a1,a2=A.shape
  out=np.zeros((a1-b1+1,a2-b2+1))
  for i in range (a1-b1+1):
    for j in range (a2-b2+1):
      out[i][j]=np.sum(A[i:i+b1,j:j+b2]*B)
  return out
# print correlate2D

# def convFull(A,B):
#   B = np.rot90(np.rot90(B))
#   (a1,a2)=A.shape
#   (b1,b2)=B.shape

scalefactor=0.66

def convolve2DFull(A, B):
    B = np.rot90(np.rot90(B))
    (b1,b2)=B.shape
    padded_A = np.pad(A, [(b1-1, b1-1), (b2-1, b2-1)], mode='constant')
    output_shape = (A.shape[0] + b1 - 1, A.shape[1] + b2 - 1)
    output = np.zeros(output_shape)

    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            output[i][j] = np.sum(padded_A[i:i+b1, j:j+b2] * B)

    return output

In [5]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
dict1=unpickle("cifar-data/cifar-10-batches-py/data_batch_1")


def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
dict1=unpickle("cifar-data/cifar-10-batches-py/data_batch_1")
dict2=unpickle("cifar-data/cifar-10-batches-py/data_batch_2")
dict3=unpickle("cifar-data/cifar-10-batches-py/data_batch_3")
dict4=unpickle("cifar-data/cifar-10-batches-py/data_batch_4")
dict5=unpickle("cifar-data/cifar-10-batches-py/data_batch_5")



xtrain= np.vstack((dict1[b"data"],dict2[b"data"],dict3[b"data"],dict4[b"data"],dict5[b"data"]))


ytrain = np.hstack((np.array(dict1[b"labels"]),np.array(dict2[b"labels"]),np.array(dict3[b"labels"]),np.array(dict4[b"labels"]),np.array(dict5[b"labels"])))



# Reshape xtrain to have one row per training example
num_examples = xtrain.shape[0]
print(xtrain.shape)

xtrain = xtrain.reshape(num_examples, 3,32,32)

print(xtrain.shape)
print(ytrain.shape)
# Normalize the input features
print(ytrain[0:50])
# Convert pixel values to floats in the range 0-
# xtrain=xtrain.astype('float32')
xtrain = xtrain.astype('float32') / 255.0

# # Compute mean and standard deviation of pixel values in training set
# mean = np.mean(xtrain, axis=(0,1,2))
# std = np.std(xtrain, axis=(0,1,2))

# # Subtract mean from each image and divide by standard deviation
# xtrain = (xtrain - mean) / std

(50000, 3072)
(50000, 3, 32, 32)
(50000,)
[6 9 9 4 1 1 2 7 8 3 4 7 7 2 9 9 9 3 2 6 4 3 6 6 2 6 3 5 4 0 0 9 1 3 4 0 3
 7 3 3 5 2 2 7 1 1 1 2 2 0]


In [6]:
def he_init(shape):
    # shape is a tuple containing (n_in, n_out)
    n_in, n_out = shape
    # He initialization
    stddev = np.sqrt(2/n_in)
    W = np.random.normal(loc=0.0, scale=stddev, size=shape)
    return W

In [7]:
class Layer:
    def __init__(self):
      # X and Y will be 1d nparrays
        self.X = None
        self.Y = None
    
    # def forwardProp(self, input):
    #     raise NotImplementedError("This function has not been implemented here")

    # def backProp(self, output_error, learning_rate):
    #     raise NotImplementedError("This function has not been implemented here")

In [8]:
import numpy as np
input = np.array([[1,2,2],[2,3,3]])
B = np.array([1,2,0])
C = np.array([1,0,1])
D = np.array([[1,2,0],[1,0,1]])
print(np.dot(input,input.T))


[[ 9 14]
 [14 22]]


In [9]:
class Flatter(Layer):
  #change is not needed here
    def __init__(self, input_shape, output_shape):
        # there must not be a change along the batch length dimension which is the zeroth dimension. 
        assert(input_shape[0] == output_shape[0])
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forwardProp(self, input):
        return np.reshape(input, self.output_shape)

    def backProp(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

In [10]:
class MaxPool2D(Layer):
  # Assuming a square pool, can be changed easily.
    def __init__(self,input_shape,pool_height,pool_width,pool_stride):
      self.pool_height=pool_height
      self.pool_width=pool_width
      self.pool_stride=pool_stride
      self.input_shape = input_shape
      self.batch_size = input_shape[0]
      self.input_depth=input_shape[1]
      self.output_shape = (self.input_shape[0],self.input_shape[1],(int)(1+((self.input_shape[2] - pool_height)//pool_stride)),(int)(1+((self.input_shape[3] - pool_width) // pool_stride)))
      

    def forwardProp(self,input):

      self.X=input

      #this assumes that Layer has been initalised before forwardProp has been called. 
      assert(input.shape == self.input_shape)

      # print("outshape is ,",self.output_shape)
      self.Y=np.zeros(self.output_shape)
      for n in range (self.batch_size):
        for d in range(self.input_depth):
            for hi in range(self.output_shape[2]):
                for wi in range(self.output_shape[3]):
                    # print("pool height and width are : ", n,d,hi,wi,self.pool_height,self.pool_width)
                    self.Y[n,d, hi, wi] = np.max(input[n,d, hi * self.pool_stride : hi * self.pool_stride + self.pool_height, wi * self.pool_stride : wi * self.pool_stride + self.pool_width]) 
      # print(self.Y.shape)
      return self.Y

    def backProp(self,output_error,learning_rate):
          assert(output_error.shape==self.output_shape)
          del_EX = np.zeros(self.input_shape)
          
          for n in range (self.batch_size):
           for d in range(self.input_depth):
                for i in range(self.output_shape[2]):
                    for j in range(self.output_shape[3]):
                        # get the index in the region i,j where the value is the maximum
                        i_max, j_max = np.where(np.max(self.X[n,d, i * self.pool_stride : i * self.pool_stride + self.pool_height, j * self.pool_stride : j * self.pool_stride + self.pool_width]) == self.X[n,d, i * self.pool_stride : i * self.pool_stride + self.pool_height, j * self.pool_stride : j * self.pool_stride + self.pool_width])
                        if(len(i_max)==0 or len(j_max)==0):
                            print(self.X)
                        i_max, j_max = i_max[0], j_max[0]
                        # only the position of the maximum element in the region i,j gets the incoming gradient, the other gradients are zero
                        del_EX[n,d, i * self.pool_stride : i * self.pool_stride + self.pool_height, j * self.pool_stride : j * self.pool_stride + self.pool_width][i_max, j_max] = output_error[n,d, i, j]
          #  print(del_EX.shape)
          assert(del_EX.shape==self.X.shape)
          return del_EX




In [11]:
class Conv2D(Layer):
    def __init__(self, input_shape, kernel_side_length, num_kernels):
      #kernel is always a sqaure , hence kernel_size tells the side length of the kernel.
      # we start by unpacking the depth, height , width contained in the input
        batch_size, input_depth, input_height, input_width = input_shape
        # output depth will be num_kernels, each kernel gives one new output layer. 
        self.num_kernels = num_kernels
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.input_depth = input_depth
        self.output_shape = (self.batch_size,num_kernels, input_height - kernel_side_length + 1, input_width - kernel_side_length + 1)
      # the first dimension tells number of kernels, the second dimension tells depth of kernel, this will be equal to input depth, the next two tell its side lengths
        self.kernels_shape = (num_kernels, input_depth, kernel_side_length, kernel_side_length)
        # stddev = np.sqrt(2.0/self.input_depth)
        stddev=0.01
        self.kernels = np.random.normal(loc=0.0, scale=stddev, size=self.kernels_shape)
        self.Bias = np.random.normal(loc=0.0, scale=stddev, size=(num_kernels, input_height - kernel_side_length + 1, input_width - kernel_side_length + 1))
        # self.kernels = np.random.randn(*self.kernels_shape)
        # self.Bias = np.random.randn(num_kernels, input_height - kernel_side_length + 1, input_width - kernel_side_length + 1)
    def forwardProp(self,input):

      #this assumes that Layer has been initalised before forwardProp has been called. 
      assert(input.shape == self.input_shape)

      self.X=input
      # self.Y = self.Bias

      bias_4d = self.Bias[np.newaxis, :, :, :]

# Repeat along the new axis
      self.Y = np.repeat(bias_4d, self.batch_size, axis=0)

      for k in range(self.batch_size):
        for i in range(self.num_kernels):
          for j in range(self.input_depth):
            # self.Y[:, i] += correlate2DValid(self.X[:, j], self.kernels[i][j])

            # need to add a an extra 0 dimension to this operation. 
            self.Y[k][i]+=correlate2DValid(self.X[k][j],self.kernels[i][j])
      assert(self.Y.shape==self.output_shape)
      return self.Y

    def backProp(self,output_error,learning_rate):
     # output_error will be a 1-d nparray  
     #diversion
     assert(output_error.shape==self.output_shape)
     del_EK = np.zeros(self.kernels_shape)
     del_EX = np.zeros(self.input_shape)

     for k in range(self.batch_size):
       for i in range(self.num_kernels):
        for j in range(self.input_depth):
        
          #need to add extra dimension here as well, moreover for kernel you have to take mean over last dimension as well. 
            del_EK[i][j]+=correlate2DValid(self.X[k][j],output_error[k][i])
            del_EX[k][j]+=convolve2DFull(output_error[k][i],self.kernels[i,j])
    #  del_EK/=self.batch_size 
    # not dividing by mean intentionally. 
    #  del_EX/=self.batch_size

          
    #  for i in range(self.input_depth):
    #    for j in range(self.num_kernels):
    #      del_EK[j][i]=correlate2DValid(self.X[i],output_error[j])
    #      del_EX[i] += convolve2DFull(output_error[j],self.kernels[j][i])

     self.kernels-=learning_rate*del_EK
     self.Bias-=learning_rate*np.sum(output_error,axis=0)
     return del_EX


In [12]:
import numpy as np
X1 = np.array([2,3])
X2 = np.array([3,4])
Y1 = np.array([1,0,1]) 
Y2 = np.array ([5,10,5])
print(np.outer(X1,Y1))
print(np.outer(X2,Y2))
X = np.array([[2,3],[3,4]])
Y = np.array([[1,0,1],[5,10,5]])
print(np.outer(X,Y))
outer_products = X[:,:,np.newaxis] * Y[:,np.newaxis,:]

# Take the mean along the first axis (rows of A)
mean_outer_product = np.mean(outer_products, axis=0)
print(mean_outer_product)
# Flatten the result into a 1D array
# result = mean_outer_product.flatten()


[[2 0 2]
 [3 0 3]]
[[15 30 15]
 [20 40 20]]
[[ 2  0  2 10 20 10]
 [ 3  0  3 15 30 15]
 [ 3  0  3 15 30 15]
 [ 4  0  4 20 40 20]]
[[ 8.5 15.   8.5]
 [11.5 20.  11.5]]


In [13]:
class Relu(Layer):
  def forwardProp(self,input):
    self.X=input
    self.Y = np.maximum(input,0)
    return self.Y
  
  def backProp(self,output_error,learning_rate):
    return np.where(self.X < 0, 0, 1)*output_error
  

In [14]:
# This is a layer without activation, to implement activation, you add another layer.
class Linear(Layer):
  def __init__(self,batch_size,inDimension,outDimension):
    # W will be 2d np array
    # self.W =he_init((inDimension, outDimension))
    mean = 0
    stddev = 0.01
# Generate array of standard normal values
    self.W = np.random.normal(loc=mean, scale=stddev, size=(inDimension,outDimension))

    # Bias will be 1-d nparray
    # diverison
    self.Bias = np.random.normal(loc=mean, scale=stddev, size=(outDimension))
    # self.Bias = 0.01*np.ones(outDimension)
  
  def forwardProp(self,input):
    self.X=input
    self.Y = np.dot(self.X,self.W)+self.Bias
    # taken into account batch_size
    return self.Y

  def backProp(self,output_error,learning_rate):
    # output_error will be a 1-d nparray  
    #diversion
    # print("outshape is",output_error.shape)
    # print("inputshape is",self.X.shape)
    outer_products = self.X[:,:,np.newaxis] * output_error[:,np.newaxis,:]
    del_EW = np.sum(outer_products, axis=0)
    assert(del_EW.shape == self.W.shape)

    
    # del_EX= np.dot(self.W.T,output_error)
    del_EX= (np.dot(output_error,self.W.T))
    assert(self.X.shape==del_EX.shape)
    # update parameters
    self.W -= learning_rate * del_EW
    self.Bias -= learning_rate * np.sum(output_error,axis=0)
    return del_EX

In [15]:
import numpy as np
def cross_entropy(y_true,y_pred):
  # print("Y_pred obtained is ",y_pred)
  max_per_row = np.max(y_pred, axis=1)

# subtract maximum element from each element in the corresponding row
  y_pred = y_pred - max_per_row[:, np.newaxis]

  # print("y_pred is ",y_wepred)
  e_sum = np.sum(np.exp(y_pred),axis=1)
  # print("e_sum is ",e_sum)
  b = np.exp(y_pred[np.arange(len(y_true)), y_true])
  # print(b.shape)
  losses= -np.log2(((b)/(e_sum))+1e-100)
  # print(losses.shape)
  # print(losses)
  return np.mean(losses)
  # Convert the true class labels to a 2D array of shape (num_samples, num_classes) using one-hot encoding
  

def cross_entropy_derivative(y_true,y_pred):

  # print("y_pred obtained at Cross_derivative : ",y_pred)
  max_per_row = np.max(y_pred, axis=1)

# subtract maximum element from each element in the corresponding row
  y_pred = y_pred - max_per_row[:, np.newaxis]
  e_sum = np.sum(np.exp(y_pred),axis=1)
  d_y = np.exp(y_pred) / e_sum[:, np.newaxis]
  for i, j in enumerate(y_true):
    d_y[i, j] -= 1
  # print("y_true is ,",y_true,"and error that I am back propogating is ",d_y)
  return d_y


In [16]:
# print(cross_entropy(np.array([2,1,4,2])),np.array([[0,0,100,0,0],[100,2,1,1,1],[0.1,0.2,0.3,0.1,0.3],[0.4,0.4,0.1,0.05,0.05]]))

In [17]:
batch=32

In [18]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forwardProp(output)
    return output


def train(network, loss, loss_prime, x_train, y_train, epochs = 1000, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0
        iter=0
        for i in tqdm(range(0,(50000//batch)*batch,batch)):
            x=x_train[i:i+batch]
            y=y_train[i:i+batch]
            # forward
            output = predict(network, x)
            iter+=1
            # error
            l = loss(y, output)
            error += l
            print("loss in iter, ",iter , " is ",l*scalefactor)
            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backProp(grad, learning_rate)

        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")


In [19]:
def mse_loss(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_loss_derivative(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [ ]:


# neural network
network = [
    Conv2D((batch,3, 32, 32),3,32),
    Relu(),
    MaxPool2D((batch,32,30,30),2,2,2),
    Conv2D((batch,32, 15, 15), 5,64),
    Relu(),
    MaxPool2D((batch,64,11,11),2,2,2),
    Conv2D((batch,64, 5, 5), 3 ,64),
    Relu(),
    Flatter((batch,64, 3, 3), (batch,576)),
    Linear(batch,576, 64),
    Relu(),
    Linear(batch,64, 10)
    # Softmax()
]

# network = [
#     # MaxPool2D((batch,3,32,32),2,2,2),
#     Flatter((batch,3, 32, 32), (batch,3072)),
#     Linear(batch,3072, 256),
#     # Relu(),
#     Linear(batch,256, 64),
#     # Relu(),
#     Linear(batch,64, 10)
#     # Softmax()
# ]

# network = [
#     MaxPool2D((batch,3,32,32),2,2,2),
#     Flatter((batch,3,16,16),(batch,768)),
#     Linear(batch,768,10),
#     Relu()
# ] 

# train
train(
    network,
    cross_entropy,
    cross_entropy_derivative,
    xtrain,
    ytrain,
    epochs=20,
    learning_rate=0.1
)

# test
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")